In [19]:
import pandas as pd
import numpy as np

from collections import defaultdict
from surprise import SVD
from surprise import SVDpp
from surprise import accuracy
from surprise import Reader, Dataset, SVD, evaluate
from surprise.model_selection import cross_validate
from surprise import BaselineOnly
from surprise.model_selection import train_test_split

In [3]:
#Read the train and test dataset
train_dataFrame = pd.read_csv('/Users/saranggrover/Desktop/yelp-dataset/train.csv')
test_dataFrame = pd.read_csv('/Users/saranggrover/Desktop/yelp-dataset/test.csv')
train_dataFrame.shape

(1076577, 12)

In [4]:
test_dataFrame.shape

(336431, 12)

In [5]:
reader = Reader()
trainedData = Dataset.load_from_df(train_dataFrame[['user_id', 'business_id', 'stars']], reader)

In [6]:
trainedDataset, testDataset = train_test_split(trainedData, test_size=0.20)

In [7]:
#Applying Baseline Model
params = {'method': 'als','n_epochs': 10,'reg_u': 5, 'reg_i': 8}
baseline = BaselineOnly(bsl_options=params)
baseline_predictions = baseline.fit(trainedDataset).test(testDataset)


Estimating biases using als...


In [8]:
accuracy.rmse(baseline_predictions)

RMSE: 1.0782


1.0781525947951247

In [9]:
cross_validate(baseline, trainedData, measures=['RMSE'], cv=5,return_train_measures=True,verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0770  1.0782  1.0779  1.0766  1.0804  1.0780  0.0013  
RMSE (trainset)   0.9911  0.9909  0.9910  0.9912  0.9902  0.9909  0.0003  
Fit time          4.58    4.99    5.02    5.01    5.05    4.93    0.18    
Test time         1.77    1.05    2.14    2.13    1.62    1.74    0.40    


{'test_rmse': array([1.07697954, 1.07816723, 1.07791488, 1.0766204 , 1.08043231]),
 'train_rmse': array([0.99110984, 0.99092616, 0.99096369, 0.99120331, 0.99021652]),
 'fit_time': (4.580384016036987,
  4.9860169887542725,
  5.021322965621948,
  5.009507656097412,
  5.0467140674591064),
 'test_time': (1.77125883102417,
  1.0483298301696777,
  2.1427419185638428,
  2.133183002471924,
  1.618299961090088)}

In [54]:
#Applying SVD
svd = SVD(n_factors=20,n_epochs=20,biased=True,init_mean=0,verbose=True)
svd.fit(trainedDataset)
predictions_svd = svd.test(testDataset)
accuracy.rmse(predictions_svd, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 1.0815


1.0815456077110763

In [53]:
#Applying SVDPP
svdpp = SVDpp(n_factors=20,n_epochs=20,init_mean=0,verbose=True)
svdpp.fit(trainedDataset)
predictions_svdpp = svd.test(testDataset)
accuracy.rmse(predictions_svdpp, verbose=True)


 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
RMSE: 1.0802


1.0801972534287216

In [15]:
#Now we have the 3 models,getting recommendations on the test Data
userArray = np.array(test_dataFrame['user_id'])
userList = userArray.tolist() 

restaurantArray = np.array(test_dataFrame['business_id'])
restaurantList = restaurantArray.tolist()

In [65]:
#Predicting ratings using the Baseline model 
baseLinePredictionsArray = []

for item in range(len(test_dataFrame)):
    baseLinePredictionsArray.append(round(baseline.predict(userList[item], restaurantList[item])[3], 2)) 
    


In [66]:
#Predicting ratings using svd
svdpredictionsArray = []

for item in range(len(test_dataFrame)):
    svdpredictionsArray.append(round(svd.predict(userList[item], restaurantList[item])[3], 2)) 
                  


In [67]:
#Predicting ratings using SVDPP
svdppPredictions = []

for item in range(len(test_dataFrame)):
    svdppPredictions.append(round(svdpp.predict(userList[item], restaurantList[item])[3], 2)) 
    


In [21]:
print(baseline_predictions[0])

user: reJ7c2yq3f8dODMslzODUQ item: 9xSwne4GjwZ6Hlzdx2Zszg r_ui = 4.00   est = 3.96   {'was_impossible': False}


In [61]:
#Custom method to get top 3 recommendations 
def top_restaurant_recommendations(predictionArray, topN = 3):
    topRecommendations = defaultdict(list)
    for user, item, r_ui, est, _ in predictionArray:
        topRecommendations[user].append((item, est))
     
    for user_id, user_ratings in topRecommendations.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        topRecommendations[userid] = user_ratings[:topN]
     
    return topRecommendations

In [62]:
topRecommendations = top_restaurant_recommendations(baseline_predictions)

In [63]:
i=0;
restaurant_list = []
for userid, user_ratings in top_recommendations.items():
    print("user", userid)
    for (itemid, _) in user_ratings:
        restaurant_list.append(itemid)
        print("item", itemid)
    i=i+1;
    if(i==1):
        break;

user reJ7c2yq3f8dODMslzODUQ
item RdUsJhX11B0gENNc4Ml5vw
item oM8GqWtU1_ld7lLnYe3KcA
item 9xSwne4GjwZ6Hlzdx2Zszg


In [64]:
for i in range(len(test_dataFrame)):
    if test_dataFrame['user_id'][i] == 'reJ7c2yq3f8dODMslzODUQ':
        print("Restaurant Recommendations for userid reJ7c2yq3f8dODMslzODUQ : ",test_dataFrame[test_dataFrame['business_id'].isin(item_list)]['name'].unique())
        break

Restaurant Recommendations for userid reJ7c2yq3f8dODMslzODUQ :  ['Sausagefest' 'Deville Dinerbar' 'Cacao 70']
